In [ ]:
from component.tile.calculation_tile import CalculationTile
from component.tile.dashboard_tile import DashboardTile
from component.tile.vegetation_tile import VegetationTile
from component.tile.aoi_tile import AoiTile

from component.widget.calc_params import Calculation
from component.model import MgciModel
from sepal_ui.aoi.aoi_model import AoiModel

In [ ]:
aoi_tile = AoiTile()
aoi_tile;

In [ ]:
model = MgciModel(AoiModel(admin="959"))

In [ ]:
vegetation_tile = VegetationTile(model=model, aoi_model=model.aoi_model)
vegetation_tile

In [ ]:
calculation_tile = CalculationTile(model=model, units="sqkm", rsa=True)
calculation_tile;

In [ ]:
import tests.test_result as test
import component.scripts as cs

In [ ]:
model.results = test.results
model.sub_a_year = test.sub_a_year
model.sub_b_year = test.sub_b_year
model.reporting_a_years = {}
model.reporting_a_years = cs.get_sub_a_break_points(model.sub_a_year)
model.same_asset_matrix = False

In [ ]:
df = cs.get_result_from_year(model, "2010_2015", "sub_b")

In [ ]:
df

In [ ]:
# group by from_lc and to_lc
df_sankey = df[df.belt_class==4].groupby(["from_lc", "to_lc"], as_index=False).sum()

In [ ]:
import plotly.graph_objects as go

# Define nodes
nodes = list(set(df_sankey['from_lc']).union(set(df_sankey['to_lc'])))
node_labels = [f'LC {i}' for i in nodes]

In [ ]:
# Define links
links = []
for i, row in df_sankey.iterrows():
    source = nodes.index(row['from_lc'])
    target = nodes.index(row['to_lc'])
    value = row['sum']
    links.append(dict(source=source, target=target, value=value))

links

In [ ]:
# Create Sankey diagram
fig = go.Figure(data=[go.Sankey(
    arrangement='fixed',
    node=dict(
        pad=50,
        thickness=20,
        line=dict(color='gray', width=0.5),
        label=node_labels,
        color='blue'
    ),
    link=dict(
        source=[link['source'] for link in links],
        target=[link['target'] for link in links],
        value=[link['value'] for link in links],
    ))])

fig.update_layout(title_text='Sankey Diagram', font_size=10)
fig.show()
